<a href="https://colab.research.google.com/github/nicobrx/prettify_gtm_export/blob/main/prettify_GTM_JSON_export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports and authentication

In [45]:
from google.colab import files
from google.colab import auth
from google.auth import default
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import json

# Authenticate and mount Google Drive
auth.authenticate_user()
creds, _ = default()

### Upload a Google Tag Manager container export file

In [46]:
# upload the GTM JSON export

upload = files.upload()
filename = list(upload.keys())[0]  # get the name of the uploaded file
with open(filename, 'r') as file:
    gtm_data = json.load(file)


Saving GTM-KBSFZN3_workspace19.json to GTM-KBSFZN3_workspace19.json


### Add trigger lookups
The export doesn't include built-in triggers definitions.
This dictionary gets added to the list from the export.

In [60]:

# need to fill out complete list
built_in_triggers = {
    '2147479553':'All Pages',
    '2147479572':'Consent Initialization'
}

tag_types = {
    'awct':'Google Ads Conversion Tracking',
    'cvt_31071642_5':'Facebook Pixel',
    'gaawe':'GA4 Event',
    'gclidw':'Conversion Linker',
    'googtag': 'Google Tag',
    'html':'Custom HTML',
    'ua':'Google Analytics: Universal Analytics'

}

### Create HTML documentation from export data

In [61]:
# Extract tags, triggers, and variables
tags = gtm_data.get('containerVersion',{}).get('tag',[])
triggers = gtm_data.get('containerVersion',{}).get('trigger',[])
variables = gtm_data.get('containerVersion',{}).get('variable',[])

# Create dictionaries to map trigger IDs to names for easier lookup
trigger_id_to_name = {trigger['triggerId']: trigger['name'] for trigger in triggers}
trigger_id_to_name.update(built_in_triggers)

# Build the HTML content
html_content = "<html><body>"

# Tags
html_content += "<h1>Tags</h1>"
html_content += "<ul>"
for tag in tags:
    tag_name = tag.get('name', 'Unknown Tag Name')
    tag_type = tag.get('type', 'Unknown Type')
    tag_type_name = tag_types.get(tag_type, tag_type)
    tag_params = tag.get('parameter', [])

    # Extract firing triggers using trigger IDs
    firing_triggers = [trigger_id_to_name.get(trigger_id, trigger_id) for trigger_id in tag.get('firingTriggerId', [])]

    html_content += f"<li><strong>{tag_name}</strong> (Type: {tag_type_name})"
    html_content += "<ul>"

    html_content += "<li><strong>Parameters:</strong><ul>"
    for param in tag_params:
        param_key = param.get('key', 'Unknown Key')
        param_value = param.get('value', 'Unknown Value')
        html_content += f"<li>{param_key}: {param_value}</li>"
    html_content += "</ul></li>"

    html_content += "<li><strong>Firing Triggers:</strong><ul>"
    for trigger_name in firing_triggers:
        html_content += f"<li>{trigger_name}</li>"
    html_content += "</ul></li>"

    html_content += "</ul></li>"
html_content += "</ul>"

# Triggers
html_content += "<h1>Triggers</h1>"
html_content += "<ul>"
for trigger in triggers:
    trigger_name = trigger.get('name', 'Unknown Trigger Name')
    trigger_type = trigger.get('type', 'Unknown Type')
    filters = trigger.get('filter', [])

    html_content += f"<li><strong>{trigger_name}</strong> (Type: {trigger_type})"

    # Check if there are any filters
    if filters:
        html_content += "<ul><li><strong>Filter Conditions:</strong><ul>"
        for f in filters:
            filter_type = f.get('type', 'Unknown Filter Type')
            parameters = f.get('parameter', [])

            # Describe each filter
            param_descriptions = {}
            for param in parameters:
                param_key = param.get('key', 'Unknown Key')
                param_value = param.get('value', 'Unknown Value')
                param_descriptions[param_key] = param_value
            param_text = param_descriptions['arg0'] + ' ' + filter_type + ' ' + param_descriptions['arg1']
            html_content += f"<li>{param_text}</li>"
        html_content += "</ul></li></ul>"

    html_content += "</li>"
html_content += "</ul>"

# Variables
html_content += "<h1>Variables</h1>"
html_content += "<ul>"
for variable in variables:
    variable_name = variable.get('name', 'Unknown Variable Name')
    variable_type = variable.get('type', 'Unknown Type')

    html_content += f"<li><strong>{variable_name}</strong> (Type: {variable_type})</li>"
html_content += "</ul>"

html_content += "</body></html>"

# Save the HTML content to a file
html_file_path = '/content/gtm_container_documentation.html'
with open(html_file_path, 'w') as html_file:
    html_file.write(html_content)




### Create a Google doc from the output

In [62]:
# Create a new Google Doc
docs_service = build('docs', 'v1', credentials=creds)
document = docs_service.documents().create(body={'title': 'GTM Container Documentation'}).execute()
doc_id = document['documentId']

# Convert HTML to Google Docs format and upload
drive_service = build('drive', 'v3', credentials=creds)
media = MediaFileUpload(html_file_path, mimetype='text/html', resumable=True)
drive_response = drive_service.files().update(
    fileId=doc_id,
    media_body=media,
    fields='id'
).execute()

doc_link = f"https://docs.google.com/document/d/{doc_id}/edit"

print(f"Google Doc Link: {doc_link}")

Google Doc Link: https://docs.google.com/document/d/1dqd5yJr9sH3bEXtz98opFUoptpPU7_wQbmFprS1gRh4/edit
